This code is an exercise of basic class uses applied to simple linear algebra operations, without consulting any books, using the most common linear algebra algorithms (so I am not super interested on performance here, but rather on the implementation exercise). It is not supposed to be comprehensive all on linear algebra operations available and it is still under development.

In [1]:
class Exceptions(Exception):
    pass

In [2]:
class Typer:
    def typer(self, args): #allows for any type of in put, be it loose numbers, lists or tuples and transforms into a list
        if type(args[0]) == list or type(args[0]) ==  tuple: 
            self.value = list(args[0])
        else:
            self.value = list(args) 
        return self.value

In [3]:
class Vector(Typer):
    def __init__(self, *args):
        self.type = "Vector"
        self.value = Typer.typer(self, args)
        self.shape = len(self.value)
    
    def __call__(self):
        return self.value
    
    def __add__(self, args):
        if not isinstance(args, Vector):  #transform object in instance if not
            args = Vector(args)
        if len(self.value) == len(args.value):
            return Vector([self.value[component]+args.value[component] for component in range(len(self.value))])
        else: #raise dimension error
            raise Exceptions("Dimensions do not match!")
    
    def __radd__(self, args):
        if not isinstance(args, Vector):  #transform object in instance if not
            args = Vector(args)
        if len(self.value) == len(args.value):
            return Vector([self.value[component]+args.value[component] for component in range(len(self.value))])
        else: #raise dimension error
            raise Exceptions("Dimensions do not match!")
    
    def Dot(self, args):
        if not isinstance(args, Vector): #transform object in instance if not
            args = Vector(args)
        if len(self.value) == len(args.value):
            return sum([self.value[component]*args.value[component] for component in range(len(self.value))])
        else: #raise dimension error
            raise Exceptions("Dimensions do not match!")
    
    def __mul__(self, args):
        if "index" in dir(args) or isinstance(args, Vector): #dot product
            return self.Dot(args)
        else:
            return Vector([self.value[component]*args for component in range(len(self.value))])
    
    def __rmul__(self, args):
        if "index" in dir(args) or isinstance(args, Vector): #dot product
            return self.Dot(args)
        else:
            return Vector([self.value[component]*args for component in range(len(self.value))])
    
    def __sub__(self, args):
        if not isinstance(args, Vector): #transform object in instance if not
            args = Vector(args)
        if len(self.value) == len(args.value):
            return self.__add__(args.__mul__(-1))
        else: #raise dimension error
            raise Exceptions("Dimensions do not match!")
        
    def Norm(self):
        return (self.Dot(self))**(1/2)
    
    def Normalizer(self):
        norm = self.Norm()
        return Vector([self.value[component]/norm for component in range(len(self.value))])
    
    def ComponentFunction(self, function):
        '''
        Apply a saclar function to all components
        '''
        return Vector([function(self.value[component]) for component in range(self.shape)])
    
    

In [4]:
class OnesVector(Vector):
    def __init__(self, length):
        value = [1 for component in range(length)]
        Vector.__init__(self, value)        

In [5]:
class ZerosVector(Vector):
    def __init__(self, length):
        value = [0 for component in range(length)]
        Vector.__init__(self, value) 

In [6]:
def Remover(list_, arg):
        if arg != "False": #I will use string "False" as False == 0 and so we would not be able to remove 0 
            list_.remove(arg)
            return list_
        else:
            return list_

In [7]:
class Matrix(ZerosVector): #I have mainly inheritted Vector for being able to use Dot easier
    def __init__(self, args):
        self.type = "Matrix"
        if type(args) == list: #allow for either a list of lists, a list of tuples, a tuple of lists or a tuple of tuples
            self.value = [list(args[row]) for row in range(len(args))]
        elif type(args) == tuple:
            self.value = [list(args[row]) for row in range(len(args))]
        self.shape = (len(self.value),len(self.value[0]))

    def __call__(self):
        return self.value
    
    def GetRow(self, row):
        return Vector(self.value[row])
    
    def GetColumn(self, column):
        return Vector([self.value[row][column] for row in range(self.shape[0])])
    
    def AddRow(self, arg, row = "Def", padding = False, pad = 0):
        '''
        Adds a row at row position or appends one if defeault. Padding optional for adding a shorter row.
        '''
        if isinstance(arg, Vector): #allows for directly adding a vector
            arg = arg.value[:] #save arg's state
        else:
            arg = list(arg)
        if len(arg) !=  self.shape[1]: #however, we cannot add a shorter row
            if padding == True:#allows padding, if desired
                for difference in range(self.shape[0]-len(arg)):
                    arg.append(pad)
            else:
                raise Exceptions("Dimensions do not match!")
                return
        if row == "Def": #makes the defeault row the last of the matrix
            row = self.shape[1]
        value = self.value[:] #copy the matrix to save state
        try: #if not out of index, we simply substitute the row by the new one
            value[row] = arg
        except IndexError: #else we append it to the end
            value.append(arg)
        return Matrix(value)
    
    def AddColumn(self, arg, column = "Def", padding = False, pad = 0):
        '''
        Adds a column at column position or appends one if defeault. Padding optional for adding a shorter column.
        '''
        if isinstance(arg, Vector): #allows for directly adding a vector
            arg = arg.value[:]
        else:
            arg = list(arg)
        if len(arg) != self.shape[0]: #however, we cannot add a shorter column
            if padding == True :#allows padding, if desired
                for difference in range(self.shape[1]-len(arg)):
                    arg.value.append(pad)
            else:
                raise Exceptions("Dimensions do not match!")
                return
        if column == "Def":  #makes the defeault row the last of the matrix
            column = self.shape[0]
        value = self.value[:] #copy the matrix to save state
        for row in range(self.shape[0]):
            try: #if not out of index, we simply substitute the column by the new one
                value[row][column] = arg[row]
            except IndexError: #else we append it to the end
                value[row].append(arg[row]) 
        return Matrix(value)
    
    def AddRows(self, args):
        '''
        Allows for adding more than one row to a matrix
        '''
        if isinstance(args, Matrix):
            args = args.value
        result = Matrix(self.value[:]) #copies the input matrix to avoid overwriting it
        for arg in args:
            result = result.AddRow(arg)
        return result
     
    def AddColumns(self, args):
        '''
        Allows for adding more than one column to a matrix
        '''
        if isinstance(args, Matrix):
            args = args.value
        result = Matrix(self.value[:]) #copies the input matrix to avoid missing it
        for arg in args:
            result = result.AddColumn(arg)
        return Matrix(result)
    
    def __add__(self, args):
        if not isinstance(args, Matrix): #transform object in instance if not
            args = Matrix(args)
        if self.shape != args.shape: #raise dimension error
            raise Exceptions("Dimensions do not match!")
        else:
            return Matrix([[self.value[row][column]+args.value[row][column] for column in range(self.shape[1])] for row in range(self.shape[0])])
     
    def __radd__(self, args):
        if not isinstance(args, Matrix): #transform object in instance if not
            args = Matrix(args)
        if self.shape != args.shape: #raise dimension error
            raise Exceptions("Dimensions do not match!")
        else:
            return Matrix([[self.value[row][column]+args.value[row][column] for column in range(self.shape[1])] for row in range(self.shape[0])]) 
    
    def __mul__(self, args): #matrix product
        if not isinstance(args, Matrix) or not isinstance(args, Vector): #transform object in instance if not
            if "index" not in dir(args): #if args is a number
                return Matrix([[self.value[row][column]*args for column in range(self.shape[1])] for row in range(self.shape[0])])
            else:
                args = Matrix(args)
        elif args.type == "Vector": #if the argument is a vector, transform it into a matrix
            args = Matrix(args.value)
        if self.shape[1]!=args.shape[0]: #raise dimension error
            raise Exceptions("Dimensions do not match! Matrix multiplication requires inputs of form (m,n) x (n,o)!")
        else:
            return Matrix([[Vector.Dot(self.GetRow(row), args.GetColumn(column)) for column in range(args.shape[1])] for row in range(self.shape[0])])   
       

    def __rmul__(self, args): #matrix product
        if not isinstance(args, Matrix) or not isinstance(args, Vector): #transform object in instance if not
            if "index" not in dir(args): #if args is a number, not a tuple or list
                return Matrix([[self.value[row][column]*args for column in range(self.shape[1])] for row in range(self.shape[0])])
            else:
                args = Matrix(args)
        elif args.type == "Vector": #if the argument is a vector, transform it into a matrix
            args = Matrix(args.value)
        if self.shape[1]!=args.shape[0]: #raise dimension error
            raise Exceptions("Dimensions do not match! Matrix multiplication requires inputs of form (m,n) x (n,o)!")
        else:
            return Matrix([[Vector.Dot(args.GetRow(row), self.GetColumn(column)) for column in range(self.shape[1])] for row in range(args.shape[0])])   
       
    def __sub__(self, args):
        if not isinstance(args, Matrix): #transform object in instance if not
            args = Matrix(args)
        if self.shape != args.shape: #raise dimension error
            raise Exceptions("Dimensions do not match!")
        else:
            return self.__add__(args.__mul__(-1))
    
    def Commutator(self, args):
        if not isinstance(args, Matrix): #transform object in instance if not
            args = Matrix(args)
        elif args.type == "Vector":
            args = Matrix(args.value)
        if self.shape[1]!=args.shape[0]: #raise dimension error
            raise Exceptions("Dimensions do not match! Matrix multiplication requires inputs of form (m,n) x (n,o)!")
        else:
            return Matrix(self.__mul__(args)-args.__mul__(self)) 
    
    def Transpose(self):
        return Matrix([[self.value[row][column] for row in range(self.shape[0])] for column in range(self.shape[1])])  
    
    def Diagonal(self):
        self.diagonal = True 
        for row in range(self.shape[0]):
            for column in range(self.shape[1]): #iterates for all components
                if column != row and self.value[row][column] !=0: #if a out of diagonla compoent is non-zero, break
                    print("Not diagonal") 
                    self.diagonal = False
                    break
            if self.diagonal == False:
                break
        else:
            print("Diagonal")
    
    def Square(self):
        if self.shape[0] == self.shape[1]:
            return True
        else:
            return False
    
    def SliceMatrix(self, row_min, row_max, column_min, column_max, remove_row = "False", remove_column = "False"):
        """
        Only split if the ranges are sliceable as in matrix.value[row_min:row_max][column_min:column_max], else returns the zero matrix
        """
        if row_min in range(self.shape[0]) and row_max in range(self.shape[0]+1) and column_min in range(self.shape[0]) and column_max in range(self.shape[0]+1) and row_min != row_max and column_min != column_max:
            range_row = Remover(list(range(row_min, row_max)), remove_row)
            range_column = Remover(list(range(column_min, column_max)), remove_column)
            return Matrix([[self.value[row][column] for column in range_column] for row in range_row])
        else:
            return Matrix([[0]])
    
    def Det(self):
        """
        Do a Lagrange's method on the first column, with potential simplification for diagonal matrices. Probably not the best
        algorithm, but definetely the one I am most used to.
        """
        if not self.Square(): #check for square matrices
            raise Exceptions("Only applicable to square matrices!")
        det = 0
        if self.shape[0] == 2:#if 2x2 matrix
            det += self.value[0][0]*self.value[1][1]-self.value[0][1]*self.value[1][0]
        elif self.shape[0] > 2: #else do the recursion
            for column in range(self.shape[1]): 
                if column % 2 == 1:
                    coef = (-1)*self.value[0][column]
                else:
                    coef = self.value[0][column]
                det += coef*(self.SliceMatrix(1, self.shape[0], 0, self.shape[1], remove_column = column)).Det()
        self.det = det
        return det

    def Invertible(self):
        return True if self.Det() != 0 and self.Square() else False
    
    def Inverse(self):
        '''
        Again, perhaps not the fastest algorithm, but the one I am most used to.
        '''
        if not self.Square():
            raise Exceptions("Only applicable to square matrices!")
        if not self.Invertible():
            raise Exceptions("Matrix is not invertible")
        det = self.Det() #calculates determinant
        coef_matrix = Matrix([[self.SliceMatrix(0, self.shape[0], 0, self.shape[1], remove_row = row, remove_column = column).Det()
                        *(-1)**(row % 2)*(-1)**(column % 2) for column in range(self.shape[1])] for row in range(self.shape[0])])
        return (1/det)*coef_matrix.Transpose()
    
    def GaussianElimination(self):
        '''
        Retruns the matrix in echelon form.
        '''
        pivot_row = self.value[0]
        echelon = self.value[:] #makes a copy so that we do not affect the initial matrix
        if len(echelon) == 1: #if we have a one row matrix, we terminate the algorihm
            return Matrix(echelon)
        pivot_value = 0
        for column in range(len(echelon[0])):
                for row in range(len(echelon)): #chooses a row that does not start with 0 to pivot, else use the first row
                    if echelon[row][column] != 0:
                        pivot_column = column
                        pivot_row = echelon[row]
                        pivot_value = echelon[row][pivot_column]
                        echelon[row] = echelon[0]
                        echelon[0] = pivot_row
                        break
                if pivot_value != 0:
                    break
        else: #a matrix of zeros
            return Matrix(echelon)
        vector_pivot = Vector(pivot_row) #makes pivot row into a vector so we can use the defined vector operations
        for row in range(1, len(echelon)): #does the actual substiution step
            leading_coef = -echelon[row][pivot_column]/pivot_row[pivot_column]
            echelon[row] = (Vector(echelon[row]).__add__(vector_pivot.__mul__(leading_coef))).value
        matrix = Matrix([pivot_row])
        matrix = matrix.AddRows(Matrix(echelon[1:]).GaussianElimination()) #recursion step
        return matrix
    
    def ReducedGaussianElimination(self):
        '''
        Returns the matrix in reduced echelon form
        '''
        echelon = self.GaussianElimination().value
        if len(echelon) == 1: #if we have a one row matrix, we terminate the algorihm
            return Matrix(echelon)
        list_of_indexes = list(range(len(echelon)))
        list_of_indexes.reverse() #start the process with the last row
        pivot_value = 0
        for row in list_of_indexes:
            for column in range(len(echelon[0])):
                if echelon[row][column] != 0: #looks for the first non-zero value in the row (i.e. the pivot value), if does not find, keep looking
                    pivot_value = echelon[row][column]
                    pivot_row = echelon[row]
                    pivot_column = column
                    break
            if pivot_value != 0:
                break
        else: #matrix of zeros
            return Matrix(echelon)
        vector_pivot = Vector(pivot_row) #makes pivot row into a vector so we can use the defined vector operations
        vector_pivot = vector_pivot.__mul__(1/pivot_value) #scales the matrix by leading (pivot) coeficient
        for row in range(0, len(echelon)-1): #does the actual substiution step for all rows above it
            leading_coef = -echelon[row][pivot_column]
            echelon[row] = Vector(echelon[row]).__add__(vector_pivot.__mul__(leading_coef)).value
        matrix = Matrix(echelon[:-1]).ReducedGaussianElimination().AddRows(Matrix([vector_pivot.value])) #must add the vector version as this is the scaled one
        value = matrix.value[:]
        for row in range(matrix.shape[0]): #it may happen that last column is not the one full of zeros, so swtich columns if that is the case
            if value[row] == ZerosVector(matrix.shape[1]).ComponentFunction(float).value: #allow to compare float to float
                row_value = value[row]
                value.pop(row)
                value.append(row_value)
                matrix = Matrix(value)
        return matrix
    
    def ComponentFunction(self, function):
        '''
        Apply a saclar function to all components
        '''
        return Matrix([[function(self.value[column][row]) for column in range(self.shape[1])] for row in range(self.shape[0])])

In [8]:
class IdentityMatrix(Matrix):
    def __init__(self, dimension):
        value = [[1 if column==row else 0 for column in range(dimension)] for row in range(dimension)]
        Matrix.__init__(self, value)

In [9]:
class DiagonalMatrix(Matrix):
    def __init__(self, *args):
        value = Typer.typer(self, args) #transform arguments in list of the non-zero components
        self.dimension = len(value)
        value = [[value[column] if column==row else 0 for column in range(self.dimension)] for row in range(self.dimension)]
        Matrix.__init__(self, value)

In [10]:
class OneMatrix(Matrix):
    def __init__(self, dimension):
        value = [[1 for column in range(dimension)] for row in range(dimension)]
        Matrix.__init__(self, value)

In [11]:
class ZeroMatrix(Matrix):
    def __init__(self, dimension):
        value = [[0 for column in range(dimension)] for row in range(dimension)]
        Matrix.__init__(self, value)

In [12]:
class VectorSet(IdentityMatrix):
    '''
    Defines a set of vector and several operations as spam, linear indendence, etc
    '''
    def __init__(self, *args):
        if not isinstance(args[0], Vector): #allows for lists of lits, tuples or vectors
            self.value = [list(args[vector]) for vector in range(len(args))] #tuples or lists
        else:
            self.value = [args[vector] for vector in range(len(args))] #vectors
        self.matrix = Matrix(self.value).Transpose() #transpose the vectors to make the matrix of vectors of linear combination x1v1+x2v2+...
        self.args = [list(arg) for arg in args]
        self.value = self.matrix.value
        self.shape = self.matrix.shape
    
    def Basis(self):
        '''
        Checks if the set of n vectors forms a basis for R**n
        '''
        if not self.Square(): #the basis set must have n vectors
            return False
        else: #must be equivalent to the identity
            reduced = self.matrix.ReducedGaussianElimination()
            if reduced.value == IdentityMatrix(self.shape[0]).ComponentFunction(float).value: #must compare float to float
                return True               
            else:
                return False
    
    def LinearCombination(self, arg):
        '''
        Determines if a vector is a linear combination of others and, if so, determines the weights
        '''
        if isinstance(arg, Vector):
            arg = arg.value
        matrix = self.matrix.AddColumn(arg) #puts the vector argument and the set into a single matrix
        reduced = matrix.ReducedGaussianElimination() #makes the matrix in echelon form
        reverted_indexes =  list(range(matrix.shape[0])) #reverts the matrix to start iterating from lowest row
        reverted_indexes.reverse()
        for row in reverted_indexes:
            if reduced.value[-1][:-1] == [float(0) for component in range(len(reduced.value[-1][:-1]))] and reduced.value[-1] !=0: #checks if there is an inconsitency of form 1==0
                raise Exceptions("Vector argument not in spam!")
                break
        else:
            variables_poisitions = set(range(self.shape[1])) 
            dependent_variables_positions = set({}) #makes a list of independent variables
            for row in range(reduced.shape[0]):
                for column in range(reduced.shape[1]):
                    if reduced.value[row][column] == float(1):
                        dependent_variables_positions.add(column)
                        break
            independent_variables_positions = variables_poisitions-dependent_variables_positions
            for row in dependent_variables_positions:
                constant = float([reduced.value[row][-1]][0])
                string = "{0} x {1}".format(constant, self.args[row]) #we use row here when we actually mean collumn because the matrix is transposed
                for independent_column in independent_variables_positions:
                    string += " + {0} x {1}".format(reduced.value[row][independent_column],self.args[independent_column])
                print(string)
            return
                